In [14]:
import os
import json
import pandas as pd
import traceback
from dotenv import load_dotenv

In [3]:
from grpc._cython import cygrpc
from grpc._typing import MetadataType
from langchain_google_genai import ChatGoogleGenerativeAI

In [15]:
load_dotenv()

True

In [16]:
os.environ["GOOGLE_API_KEY"] = GOOGLE_API_KEY

In [13]:
llm = ChatGoogleGenerativeAI(model="gemini-pro",temperature=0.7)

In [18]:
llm

ChatGoogleGenerativeAI(model='gemini-pro', client= genai.GenerativeModel(
   model_name='models/gemini-pro',
   generation_config={}.
   safety_settings={}
))

In [19]:
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain
from langchain.chains import SequentialChain
import PyPDF2

In [20]:
RESPONSE_JSON = {
    "1":{
        "mcq":"multiple choice question",
        "options":{
            "a": "choice here",
            "b": "choice here",
            "c": "choice here",
            "d": "choice here",
        },
        "correct":"correct answer",
    },
    "2":{
        "mcq":"multiple choice question",
        "options":{
            "a": "choice here",
            "b": "choice here",
            "c": "choice here",
            "d": "choice here",
        },
        "correct":"correct answer",
    },
    "3":{
        "mcq":"multiple choice question",
        "options":{
            "a": "choice here",
            "b": "choice here",
            "c": "choice here",
            "d": "choice here",
        },
        "correct":"correct answer",
    },
}

In [69]:
TEMPLATE="""
Text:{text}
You are an expert MCQ maker, given the above text, it is your job to \
create a quiz of {number} multiple choice questions for {subject} students in {tone} tone.
Make sure the questions are not repeated and check all the questions to be confirming the text as well.
Make sure to format your response like RESPONSE_JSON below and use it as a guide. \
Ensure to make {number} MCQ's
### RESPONSE_JSON
{response_json}

"""

In [70]:
quiz_generation_prompt= PromptTemplate(
    input_variables=["text", "number", "subject", "tone", "response_json"],
    template=TEMPLATE
)

In [71]:
quiz_chain = LLMChain(llm=llm, prompt=quiz_generation_prompt, output_key="quiz", verbose=True)

In [72]:
TEMPLATE2 = """"
You are an expert english grammarian and writer. Given a Multiple Choice Quiz for {Subject} students. You need to evaluate the complexity of the question and give a complete analysis of the quiz.
Only use at max 50 words for complexity if the quiz is not at per with the cognitive and analytical abilities of the students, update the quiz questions which needs to be changed and charge the
tone such that it perfectly fits the student abilities
Quiz_MCQs:
{quiz}

Check from an expert English writer of the above quiz:
"""

In [73]:
quiz_evaluation_prompt = PromptTemplate(input_variables=["subject", "quiz"], template=TEMPLATE)

In [74]:
review_chain=LLMChain(llm=llm, prompt=quiz_evaluation_prompt, output_key="review", verbose=True)

In [75]:
generate_evaluate_chain= SequentialChain(chains=[quiz_chain, review_chain],
                                         input_variables=["text", "number", "subject", "tone", "response_json"],
                                         output_variables=["quiz", "review"], verbose=True,)

In [76]:
file_path= r"C:\Users\Nikhil\OneDrive\Documents\Desktop\openai\MCQGEN\data.txt"

In [77]:
with open(file_path, 'r') as file:
    TEXT = file.read()

In [78]:
print(TEXT)

The term machine learning was coined in 1959 by Arthur Samuel, an IBM employee and pioneer in the field of computer gaming and artificial intelligence.[9][10] The synonym self-teaching computers was also used in this time period.[11][12]

Although the earliest machine learning model was introduced in the 1950s when Arthur Samuel invented a program that calculated the winning chance in checkers for each side, the history of machine learning roots back to decades of human desire and effort to study human cognitive processes.[13] In 1949, Canadian psychologist Donald Hebb published the book The Organization of Behavior, in which he introduced a theoretical neural structure formed by certain interactions among nerve cells.[14] Hebb's model of neurons interacting with one another set a groundwork for how AIs and machine learning algorithms work under nodes, or artificial neurons used by computers to communicate data.[13] Other researchers who have studied human cognitive systems contributed

In [79]:
json.dumps(RESPONSE_JSON)

'{"1": {"mcq": "multiple choice question", "options": {"a": "choice here", "b": "choice here", "c": "choice here", "d": "choice here"}, "correct": "correct answer"}, "2": {"mcq": "multiple choice question", "options": {"a": "choice here", "b": "choice here", "c": "choice here", "d": "choice here"}, "correct": "correct answer"}, "3": {"mcq": "multiple choice question", "options": {"a": "choice here", "b": "choice here", "c": "choice here", "d": "choice here"}, "correct": "correct answer"}}'

In [80]:
NUMBER=5
SUBJECT="Machine Learning"
TONE="simple"

In [81]:
response= generate_evaluate_chain(
    {
        "text": TEXT,
        "number": NUMBER,
        "subject": SUBJECT,
        "tone": TONE,
        "response_json": json.dumps(RESPONSE_JSON)
    }
)



> Entering new SequentialChain chain...


> Entering new LLMChain chain...
Prompt after formatting:

Text:The term machine learning was coined in 1959 by Arthur Samuel, an IBM employee and pioneer in the field of computer gaming and artificial intelligence.[9][10] The synonym self-teaching computers was also used in this time period.[11][12]

Although the earliest machine learning model was introduced in the 1950s when Arthur Samuel invented a program that calculated the winning chance in checkers for each side, the history of machine learning roots back to decades of human desire and effort to study human cognitive processes.[13] In 1949, Canadian psychologist Donald Hebb published the book The Organization of Behavior, in which he introduced a theoretical neural structure formed by certain interactions among nerve cells.[14] Hebb's model of neurons interacting with one another set a groundwork for how AIs and machine learning algorithms work under nodes, or artificial neurons used 

In [82]:
response

{'text': 'The term machine learning was coined in 1959 by Arthur Samuel, an IBM employee and pioneer in the field of computer gaming and artificial intelligence.[9][10] The synonym self-teaching computers was also used in this time period.[11][12]\n\nAlthough the earliest machine learning model was introduced in the 1950s when Arthur Samuel invented a program that calculated the winning chance in checkers for each side, the history of machine learning roots back to decades of human desire and effort to study human cognitive processes.[13] In 1949, Canadian psychologist Donald Hebb published the book The Organization of Behavior, in which he introduced a theoretical neural structure formed by certain interactions among nerve cells.[14] Hebb\'s model of neurons interacting with one another set a groundwork for how AIs and machine learning algorithms work under nodes, or artificial neurons used by computers to communicate data.[13] Other researchers who have studied human cognitive system

In [83]:
quiz=response.get("quiz")

In [91]:
quiz = quiz.replace('### RESPONSE_JSON\n', '')

In [92]:
print(quiz)

{"1": {"mcq": "Who coined the term 'machine learning'?", "options": {"a": "Donald Hebb", "b": "Arthur Samuel", "c": "Walter Pitts", "d": "Raytheon Company"}, "correct": "b"}, "2": {"mcq": "What was the earliest theoretical model for machine learning based on?", "options": {"a": "Neural networks", "b": "Reinforcement learning", "c": "Cognitive processes", "d": "Supervised learning"}, "correct": "a"}, "3": {"mcq": "What was the purpose of the 'goof' button in the Cybertron learning machine?", "options": {"a": "To reset the machine", "b": "To re-evaluate incorrect decisions", "c": "To store training data", "d": "To adjust the learning rate"}, "correct": "b"}, "4": {"mcq": "According to Tom M. Mitchell's definition, what is the measure of improvement in machine learning?", "options": {"a": "Accuracy", "b": "Experience", "c": "Time", "d": "Memory"}, "correct": "a"}, "5": {"mcq": "What are the two main objectives of modern-day machine learning?", "options": {"a": "Data classification and pre

In [94]:
quiz=json.loads(quiz)

In [97]:
quiz_table_data= []
for key, val in quiz.items():
    mcq= val["mcq"]
    options = " | ".join(
        [
            f"{option}: {option_value}"
            for option, option_value in val["options"].items()
        ]
    )
    correct = val["correct"]
    quiz_table_data.append({"MCQ": mcq, "Choices": options, "Correct": correct})

In [100]:
quiz_table_data

[{'MCQ': "Who coined the term 'machine learning'?",
  'Choices': 'a: Donald Hebb | b: Arthur Samuel | c: Walter Pitts | d: Raytheon Company',
  'Correct': 'b'},
 {'MCQ': 'What was the earliest theoretical model for machine learning based on?',
  'Choices': 'a: Neural networks | b: Reinforcement learning | c: Cognitive processes | d: Supervised learning',
  'Correct': 'a'},
 {'MCQ': "What was the purpose of the 'goof' button in the Cybertron learning machine?",
  'Choices': 'a: To reset the machine | b: To re-evaluate incorrect decisions | c: To store training data | d: To adjust the learning rate',
  'Correct': 'b'},
 {'MCQ': "According to Tom M. Mitchell's definition, what is the measure of improvement in machine learning?",
  'Choices': 'a: Accuracy | b: Experience | c: Time | d: Memory',
  'Correct': 'a'},
 {'MCQ': 'What are the two main objectives of modern-day machine learning?',
  'Choices': 'a: Data classification and prediction | b: Pattern recognition and reinforcement learnin

In [102]:
quiz=pd.DataFrame(quiz_table_data)

In [103]:
quiz.to_csv("Machinelearning.csv", index=False)